J'essaie de scraper des infos sur la page de présentation Fragantica de Black Phantom : https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html

Je veux scraper ces infos :
Nom du Parfum
Nom de la marque
Launch year
Perfume rating
Nombre de votant sur le Perfume rating
Saisonnalité (Hiver, automne… peut etre gardé juste la + votée)
Parfumeur
Main accord
‘Perfume Pyramide’  = [‘Top note’ , ‘Middle note’, Base note’]
Genre (en prenant celui qui à le plus de vote)
Longevite, Sillage (en prenant les votes)


## Imports

In [20]:
import requests
from bs4 import BeautifulSoup
import re
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import cloudscraper

import numpy as np


## Les fonctions

### fonction pour récuperer le code html d'une page parfum

In [37]:
#Récupère le HTML de la page correspondant à l'URL donnée.
def get_html_non_headless(url):
    """
    Ouvre Chrome en mode normal (non-headless), va sur l'URL, 
    attend 1 seconde, récupère le HTML et ferme le navigateur.
    """
    # 1) Configuration basique du driver, sans headless
    options = webdriver.ChromeOptions()
    # Pas d'options.add_argument('--headless'), car on veut voir la fenêtre

    # 2) Création du driver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    
    # 3) Accès à la page
    driver.get(url)
    
    # 4) Patiente un peu pour laisser la page (et scripts éventuels) se charger
    time.sleep(1)
    
    # 5) Récupère le code HTML
    html_content = driver.page_source
    
    # 6) Ferme le navigateur
    driver.quit()

    return html_content

### Fonctions pour récuperer les infos sur un parfum

In [41]:
# 1) Nom du parfum
def extract_perfume_name(html_content):
    """Extrait le nom du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', html_content)
    return match.group(1).replace('-', ' ') if match else None

# 2) Marque
def extract_brand_name(html_content):
    """Extrait la marque du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/([^/]+)/', html_content)
    return match.group(1).replace('-', ' ') if match else None

#3) Parfumeur
def extract_nose(soup):
    """Extrait le nom du parfumeur"""
    nose_el = soup.select_one('div.cell a[href^="/noses/"]')
    return nose_el.get_text(strip=True) if nose_el else None

#4) Année de sortie
def extract_launch_year(soup):
    """Extrait l'année de sortie du parfum"""
    title_el = soup.find('title')
    if title_el:
        possible_year = title_el.get_text(strip=True)[-4:]
        if possible_year.isdigit():
            return possible_year
    return None

#5) Perfume rating
def extract_rating(soup):
    """Extrait la note du parfum"""
    rating_el = soup.select_one('span[itemprop="ratingValue"]')
    return rating_el.get_text(strip=True) if rating_el else None

#6) Nombre de votes
def extract_rating_count(soup):
    """Extrait le nombre de votes"""
    rating_count_el = soup.select_one('span[itemprop="ratingCount"]')
    return rating_count_el.get_text(strip=True) if rating_count_el else None

#7) Accords principaux
def extract_main_accords(soup):
    """Extrait les accords principaux du parfum"""
    main_accords_el = soup.find_all('div', class_='cell accord-box')
    return [element.get_text(strip=True) for element in main_accords_el if element.get_text(strip=True)]

#8) Genre
def extract_gender(gender_list):
    """Détermine le genre du parfum en fonction des votes"""
    if not gender_list:
        return None
    
    sum_votes = np.sum([int(element[1]) for element in gender_list])
    
    if sum_votes > 8:
        female_count = int(gender_list[0][1]) + int(gender_list[1][1])
        male_count = int(gender_list[3][1]) + int(gender_list[4][1])
        unisex_count = 1.2 * int(gender_list[2][1])

        if female_count > male_count and female_count > unisex_count:
            return "female"
        elif male_count > unisex_count:
            return "male"
        else:
            return "unisex"
    return None

#9) Longévité
def extract_longevity(longevity_list):
    """Détermine la longévité du parfum"""
    if not longevity_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in longevity_list])
    if sum_votes > 8:
        longevity = np.argmax([int(element[1]) for element in longevity_list])
        return longevity_list[longevity][0]
    return None

#10) Sillage
def extract_sillage(sillage_list):
    """Détermine le sillage du parfum"""
    if not sillage_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in sillage_list])
    if sum_votes > 8:
        sillage = np.argmax([int(element[1]) for element in sillage_list])
        return sillage_list[sillage][0]
    return None

#11) Ressenti du prix
def extract_price_feeling(price_feeling_list):
    """Détermine le ressenti du prix du parfum"""
    if not price_feeling_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in price_feeling_list])
    if sum_votes > 8:
        price_feeling = np.argmax([int(element[1]) for element in price_feeling_list])
        return price_feeling_list[price_feeling][0]
    return None

#12) Pyramide olfactive

### Fonctions intermediaire pour le tableau avec Gender, Sillage, loongevity et Price feeling

In [42]:
def extract_votes(soup):
    """Extrait les catégories et leurs votes à partir du HTML."""
    tableau_votes = []
    tab_rows = soup.select('div.grid-x.grid-margin-x')  # Sélection des blocs contenant les votes
    
    for row in tab_rows:
        category_el = row.select_one('span.vote-button-name')  # Nom de la catégorie
        votes_el = row.select_one('span.vote-button-legend')  # Nombre de votes

        if category_el and votes_el:
            category = category_el.get_text(strip=True)
            votes = votes_el.get_text(strip=True)

            # Vérifier si le nombre de votes est bien un chiffre
            if votes.isdigit():
                tableau_votes.append((category, votes))

    return tableau_votes

def filter_votes(tableau_votes, labels, remove_first=False, remove_last=False):
    """
    Filtre les votes selon un ensemble de labels et enlève potentiellement le premier ou dernier élément.
    
    :param tableau_votes: Liste des votes [(catégorie, nombre)]
    :param labels: Ensemble des labels à filtrer
    :param remove_first: Supprime le premier élément si True
    :param remove_last: Supprime le dernier élément si True
    :return: Liste filtrée et éventuellement modifiée
    """
    filtered_list = [element for element in tableau_votes if element[0] in labels]

    if remove_first and filtered_list:
        filtered_list.pop(0)
    if remove_last and filtered_list:
        filtered_list.pop()
    
    return filtered_list

## Le script

### Obtenir le contenu html de la page d'Angel Share

In [8]:
url = "https://www.fragrantica.com/perfume/By-Kilian/Angels-Share-62615.html"
html_content = get_html_non_headless(url)
    
# Vous pouvez ensuite parser le HTML avec BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')


### Un script utilisant les fonctions

In [44]:
data = {}

tableau_votes = extract_votes(soup)

# Définition des groupes
longevity_labels = {"very weak", "weak", "moderate", "long lasting", "eternal"}
sillage_labels = {"intimate", "moderate", "strong", "enormous"}
gender_labels = {"female", "more female", "unisex", "more male", "male"}
price_feeling_labels = {"way overpriced", "overpriced", "ok", "good value", "great value"}

# Séparation des données avec gestion des exclusions spécifiques
longevity_list = filter_votes(tableau_votes, longevity_labels, remove_last=True)  # Supprime le dernier élément
sillage_list = filter_votes(tableau_votes, sillage_labels, remove_first=True)  # Supprime le premier élément
gender_list = filter_votes(tableau_votes, gender_labels)
price_feeling_list = filter_votes(tableau_votes, price_feeling_labels)

data = {
    "nom_parfum": extract_perfume_name(html_content),
    "marque": extract_brand_name(html_content),
    "nose": extract_nose(soup),
    "launch_year": extract_launch_year(soup),
    "rating_value": extract_rating(soup),
    "rating_count": extract_rating_count(soup),
    "main_accords": extract_main_accords(soup),
    "gender": extract_gender(gender_list),
    "longevity": extract_longevity(longevity_list),
    "sillage": extract_sillage(sillage_list),
    "price_feeling": extract_price_feeling(price_feeling_list),
}

print(data)

{'nom_parfum': 'Angels Share', 'marque': 'By Kilian', 'nose': 'Benoist Lapouza', 'launch_year': '2020', 'rating_value': '4.38', 'rating_count': '15,727', 'main_accords': ['woody', 'warm spicy', 'sweet', 'vanilla', 'cinnamon', 'amber', 'powdery'], 'gender': 'unisex', 'longevity': 'long lasting', 'sillage': 'moderate', 'price_feeling': 'overpriced'}


### Un script sans les fonctions

- nom_parfum
      - marque
      - launch_year
      - perfume_rating
      - nb_votes
      - parfumeur
      - main_accord
      - perfume_pyramid (dict avec top/middle/base)
      - genre
      - longevite_votes
      - sillage_votes


In [ ]:
data = {}

# 1) Nom du parfum        
match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', html_content)
if match:
    perfume_name = match.group(1)
    perfume_name = perfume_name.replace('-', ' ')
    data['nom_parfum'] = perfume_name
else:
    data['nom_parfum'] = None

#2) Marque
match = re.search(r'/perfume/([^/]+)/', html_content)
if match:
    brand_name = match.group(1)
    brand_name = brand_name.replace('-', ' ')
    data['marque'] = brand_name
else:
    data['marque'] = None

#3) Parfumeur
data['nose'] = None
nose_el = soup.select_one('div.cell a[href^="/noses/"]')
if nose_el:
    data['nose'] = nose_el.get_text(strip=True)

#4) Année de sortie
data['launch_year'] = None
title_el = soup.find('title')
if title_el:
    possible_year = title_el.get_text(strip=True)[-4:] #Les 4 derniers caractères du titre
    if possible_year.isdigit():
        data['launch_year'] = possible_year

#5) Perfume rating (note)
## ATTENTION, valeur en string, pas en float
data['rating_value'] = None
rating_el = soup.select_one('span[itemprop="ratingValue"]').get_text(strip=True)
if rating_el:
    data['rating_value'] = rating_el  

#6) Nb de votes
data['rating_count'] = None
rating_count_el = soup.select_one('span[itemprop="ratingCount"]').get_text(strip=True)
if rating_count_el:
    data['rating_count'] = rating_count_el


#7)Main accord
data['main_accords'] = []
main_accords_el = soup.find_all('div', class_= 'cell accord-box')
for element in main_accords_el:
    accord = element.get_text(strip=True)
    if accord:
        data['main_accords'].append(accord)

#8) Gender
##Regle de calcul : max ( female + more female , male + more male , 1,2* unisex)
data['gender'] = None
sum_votes = np.sum([int(element[1]) for element in gender_list])

if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None

    female_count = int(gender_list[0][1]) + int(gender_list[1][1])
    male_count = int(gender_list[3][1]) + int(gender_list[4][1])
    unisex_count = 1.2 * int(gender_list[2][1])
    
    if female_count > male_count and female_count > unisex_count:
        data['gender'] = "female"
    else:
        if male_count > unisex_count:
            data['gender'] = "male"
        else :
            data['gender'] = "unisex"

#9) Longévité
data['longevity'] = None
sum_votes = np.sum([int(element[1]) for element in longevity_list])
if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None
    longevity = np.argmax([int(element[1]) for element in longevity_list])
    data['longevity'] = longevity_list[longevity][0]

#10) Sillage
data['sillage'] = None
sum_votes = np.sum([int(element[1]) for element in sillage_list])
if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None
    sillage = np.argmax([int(element[1]) for element in sillage_list])
    data['sillage'] = sillage_list[sillage][0]
    
#11) Price feeling
data['price_feeling'] = None
sum_votes = np.sum([int(element[1]) for element in price_feeling_list])
if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None
    price_feeling = np.argmax([int(element[1]) for element in price_feeling_list])
    data['price_feeling'] = price_feeling_list[price_feeling][0]

print(data)


{'nom_parfum': 'Angels Share', 'marque': 'By Kilian', 'nose': 'Benoist Lapouza', 'launch_year': '2020', 'rating_value': '4.38', 'rating_count': '15,727', 'main_accords': ['woody', 'warm spicy', 'sweet', 'vanilla', 'cinnamon', 'amber', 'powdery'], 'gender': 'unisex'}


In [ ]:
#Les listes des votes pour Longevity, Sillage, Gender, Price Feeling
longevity_list = []
sillage_list = []
gender_list = []
price_feeling_list = []
#Sélecteur des blocs qui contiennent la catégorie + nombre de votes
tab_rows = soup.select('div.grid-x.grid-margin-x')  #On selectionne trop de chose donc on va devoir filtrer
tableau_votes = []
for row in tab_rows:
    # Récupérer le nom de la catégorie (ex: "female") dans <span class="vote-button-name">
    category_el = row.select_one('span.vote-button-name')
    # Récupérer le nombre de votes dans <span class="vote-button-legend">
    votes_el = row.select_one('span.vote-button-legend')
    if category_el and votes_el:
        category = category_el.get_text(strip=True)
        votes = votes_el.get_text(strip=True)   
        tableau_votes.append((category, votes))

for element in tableau_votes:
    if not element[1].isdigit(): #Si le nombre de votes n'est pas un nombre, et oui ça arrive !
        tableau_votes.remove(element)

# Définition des groupes
longevity_labels = {"very weak", "weak", "moderate", "long lasting", "eternal"}
sillage_labels = {"intimate", "moderate", "strong", "enormous"}
gender_labels = {"female", "more female", "unisex", "more male", "male"}
price_feeling_labels = {"way overpriced", "overpriced", "ok", "good value", "great value"}
# Séparation des données
longevity_list = [element for element in tableau_votes if element[0] in longevity_labels]
longevity_list.pop() #On enlève le dernier élément qui est le modérate sillage
sillage_list = [element for element in tableau_votes if element[0] in sillage_labels]
sillage_list.pop(0) #On enlève le premier élément qui est le modérate longevity
gender_list = [element for element in tableau_votes if element[0] in  gender_labels]
price_feeling_list = [element for element in tableau_votes if element[0] in  price_feeling_labels]




[('very weak', '113'), ('weak', '342'), ('moderate', '2458'), ('long lasting', '4827'), ('eternal', '1399')]
[('intimate', '421'), ('moderate', '4052'), ('strong', '3606'), ('enormous', '798')]
